## Librerias

In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import json
# import plotly.express as px
# import seaborn as sbs

import warnings
warnings.filterwarnings("ignore")

## Parametros

In [2]:
client_bq = bigquery.Client()

In [52]:
main_table = 'co-keralty-costomedico.Diabetes_avicena_survival.diabetes_3_annos'

query_for_t0_pop = f"""
SELECT numero_identificacion_paciente,
  year,
  month,
  fecha,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes FROM (SELECT
  numero_identificacion_paciente,
  year,
  month,
  DATE(CONCAT(year,'-',month,'-01')) AS fecha,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes,
FROM
  `{main_table}`
WHERE
  peso IS NOT NULL
  AND talla IS NOT NULL
  AND genero_paciente IS NOT NULL
  -- AND raza_paciente IS NOT NULL
  -- AND nivel_academico_paciente IS NOT NULL
  AND HDL IS NOT NULL
  AND LDL IS NOT NULL
  AND trigliceridos IS NOT NULL
  -- AND perimetro_abdominal IS NOT NULL
  -- AND hace_ejercicio IS NOT NULL
  AND diabetes = 0)
  WHERE orden = 1
"""

select_pob = f"""
SELECT DISTINCT numero_identificacion_paciente FROM
({query_for_t0_pop})

"""

query_for_t1_pop = """

WITH
  no_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) DESC) AS orden
    FROM
      `{}`
    WHERE
      diabetes = 0
      AND numero_identificacion_paciente IN ({}))
  WHERE
    orden = 1),

si_diabetes_t1 AS (
  SELECT
    numero_identificacion_paciente,
    year,
    month,
    DATE(CONCAT(year,'-',month,'-01')) AS fecha,
    edad,
    peso,
    talla,
    peso/(talla*talla) AS IMC,
    HDL,
    LDL,
    trigliceridos,
    perimetro_abdominal,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
    ant_cardio,
    med_hipertension,
    ant_familiar_dm,
    hace_ejercicio,
    diabetes,
  FROM (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden
    FROM
      `{}`
    WHERE
      diabetes = 1
      AND numero_identificacion_paciente IN ({}))
  WHERE
    orden = 1)

    
SELECT
  *
FROM
  no_diabetes_t1
WHERE
  no_diabetes_t1.numero_identificacion_paciente NOT IN (
  SELECT
    DISTINCT numero_identificacion_paciente
  FROM
    si_diabetes_t1)
UNION ALL (
  SELECT
    *
  FROM
    si_diabetes_t1)

"""

In [53]:
print(query_for_t0_pop)


SELECT numero_identificacion_paciente,
  year,
  month,
  fecha,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes FROM (SELECT
  numero_identificacion_paciente,
  year,
  month,
  DATE(CONCAT(year,'-',month,'-01')) AS fecha,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY DATE(CONCAT(year,'-',month,'-01')) ASC) AS orden,
  edad,
  peso,
  talla,
  peso/(talla*talla) AS IMC,
  HDL,
  LDL,
  trigliceridos,
  perimetro_abdominal,
  genero_paciente,
  raza_paciente,
  nivel_academico_paciente,
  ant_cardio,
  med_hipertension,
  ant_familiar_dm,
  hace_ejercicio,
  diabetes,
FROM
  `co-keralty-costomedico.Diabetes_avicena_survival.diabetes_3_annos`
WHERE
  peso IS NOT NULL
  AND talla IS NOT NULL
  AND genero_paciente IS NOT NULL
  -- AND raza_paciente IS NOT NU

## Funciones

In [25]:
def load_data(client, query, local = False, save = True, file_name = 'data'):

    if local:
        data = pd.read_csv(f'data/{file_name}.csv', sep = '|')
    else:
        data = client.query(query).result().to_dataframe()

    if save:
        data.to_csv(f'data/{file_name}.csv', sep = '|')

    return data

def etiquetar_diabetes(t0,t1):
    if t1 > t0:
        return t1
    else:
        return t0

## Proceso

In [54]:
fechas_minimas = client_bq.query(f'SELECT DISTINCT year, month FROM `{main_table}` ORDER BY 1, 2 LIMIT 1').result().to_dataframe()
min_year = fechas_minimas['year'].values[0]
min_month = fechas_minimas['month'].values[0]

print(f'El tiempo inicial del estudio es {min_year}-{min_month}')

El tiempo inicial del estudio es 2022-2


In [55]:
# Carga de afiliados con datos en t0
data_t0 = load_data(client=client_bq, query=query_for_t0_pop, file_name='data_t0')
print(f'Se cuentan con {data_t0.shape[0]} afiliados con datos en el momento t0')
display(data_t0.head())

# Carga de afiliados que cumplen el evento (Diabetes)
# afiliados_con_t0 = tuple(data_t0.numero_identificacion_paciente.unique())
data_t1 = load_data(client=client_bq, query=query_for_t1_pop.format(main_table,select_pob,main_table,select_pob), file_name='data_t1')
print(f'Se cuentan con {data_t1.shape[0]} afiliados que cumplen el evento (Diabetes) momento t1')
display(data_t1.head())

Se cuentan con 298837 afiliados con datos en el momento t0


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,1032383313,2025,2,2025-02-01,38,63.000000000,1.520000000,27.268005540,60.0,109.0,74.0,None,Femenino,Raizales,Básica secundaria,0,0,0,None,0
1,1090451205,2025,2,2025-02-01,33,103.500000000,1.760000000,33.412964876,45.6,109.8,223.0,109.000000000,Masculino,Otros,Ninguno,0,0,0,20 minutos\n,0
2,1092339866,2025,2,2025-02-01,37,65.000000000,1.620000000,24.767565920,56.0,85.0,57.0,None,Femenino,Otros,Básica secundaria,0,0,0,None,0
3,1117552596,2025,2,2025-02-01,26,78.000000000,1.700000000,26.989619377,27.0,80.0,292.0,None,Femenino,Otros,Básica secundaria,0,0,0,None,0
4,22642146,2025,2,2025-02-01,46,73.500000000,1.600000000,28.710937500,57.0,130.0,125.0,None,Femenino,Otros,Ninguno,0,0,0,None,0


Se cuentan con 298837 afiliados que cumplen el evento (Diabetes) momento t1


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,27610862,2022,12,2022-12-01,87,64.000000000,1.470000000,29.617289093,NaN,NaN,NaN,None,Femenino,Otros,Básica primaria,0,0,0,None,1
1,41377330,2022,12,2022-12-01,75,None,None,None,NaN,NaN,NaN,None,Femenino,Otros,Bachillerato técnico,1,1,0,None,1
2,66770252,2022,12,2022-12-01,49,79.700000000,1.600000000,31.132812500,NaN,NaN,NaN,None,Femenino,Otros,Ninguno,1,0,0,None,1
3,72281073,2022,12,2022-12-01,39,90.000000000,1.790000000,28.089010955,NaN,NaN,NaN,90.000000000,Masculino,Otros,Profesional,0,0,0,20 minutos\n,1
4,1061715763,2022,12,2022-12-01,33,53.200000000,1.590000000,21.043471382,NaN,NaN,NaN,None,Femenino,Otros,Profesional,0,0,0,None,1


In [30]:
len(data_t0.numero_identificacion_paciente.unique())

279975

In [56]:
afiliados_with_no_t1 = list(data_t0[data_t0.fecha >= data_t1.fecha].numero_identificacion_paciente.unique())

data_t1_clean = data_t1[~data_t1.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t0_clean = data_t0[~data_t0.numero_identificacion_paciente.isin(afiliados_with_no_t1)]
data_t1_clean_to_merge = data_t1_clean[['numero_identificacion_paciente','diabetes']].rename(columns = {'diabetes':'diabetes_t1'})
full_data = data_t0_clean.merge(data_t1_clean_to_merge, on = 'numero_identificacion_paciente', how = 'left') #pd.concat([data_t0_clean,data_t1_clean])
full_data.diabetes = full_data.apply(lambda x: etiquetar_diabetes(x.diabetes, x.diabetes_t1), axis = 1)
full_data = full_data.drop(columns = 'diabetes_t1')

full_data

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
0,42995991,2022,2,2022-02-01,63,64.300000000,1.630000000,24.201136663,66.0,166.50,141.00,78.000000000,Femenino,Otros,Ninguno,1,1,0,None,0
1,44155333,2022,2,2022-02-01,40,75.900000000,1.490000000,34.187649205,29.5,64.30,478.00,88.000000000,Femenino,Otros,Profesional,1,1,0,None,0
2,447022,2022,2,2022-02-01,70,87.000000000,1.770000000,27.769797951,49.0,113.80,286.00,None,Masculino,Otros,Ninguno,1,1,0,None,0
3,45452118,2022,2,2022-02-01,60,60.000000000,1.600000000,23.437500000,38.0,125.00,132.00,None,Femenino,Otros,Ninguno,0,0,0,None,0
4,45478698,2022,2,2022-02-01,56,73.000000000,1.630000000,27.475629493,72.3,59.76,73.20,94.000000000,Femenino,Otros,Normalista,1,0,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281985,63323735,2025,1,2025-01-01,60,78.900000000,1.620000000,30.064014632,55.9,71.50,143.00,98.000000000,Femenino,Otros,Profesional,0,0,0,None,1
281986,63465539,2025,1,2025-01-01,52,74.000000000,1.550000000,30.801248699,41.6,163.60,164.00,None,Femenino,Otros,Profesional,0,0,0,None,0
281987,79511536,2025,1,2025-01-01,55,57.900000000,1.650000000,21.267217631,64.8,183.60,114.82,72.000000000,Masculino,Otros,Básica secundaria,0,0,0,None,1
281988,79742743,2025,1,2025-01-01,49,70.700000000,1.706666667,24.272918694,32.8,197.00,142.00,80.000000000,Masculino,Otros,Ninguno,0,0,0,None,0


In [54]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes


In [55]:
full_data[full_data.numero_identificacion_paciente == '79569003'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
5918,79569003,2023,9,2023-09-01,52,93.500000000,1.870000000,45.0,106.0,124.0,97.000000000,Masculino,Otros,Básica secundaria,1,0,0,Nunca\n,1


In [57]:
fechas = data_t1_clean[['numero_identificacion_paciente','year','month','fecha']].rename(columns = {'year':'year_t1','month':'month_t1'})

fechas['year_event'] = fechas.fecha.apply(lambda x: x.year)
fechas['month_event'] = fechas.fecha.apply(lambda x: x.month)
fechas.head(1)

,numero_identificacion_paciente,year_t1,month_t1,fecha,year_event,month_event
0,27610862,2022,12,2022-12-01,2022,12


In [58]:
times_to_event = data_t0_clean.sort_values(['numero_identificacion_paciente','year','month'])[['numero_identificacion_paciente','year','month','diabetes']]
times_to_event = times_to_event.merge(fechas, on = 'numero_identificacion_paciente', how = 'left')
times_to_event['time_to_event'] = abs((times_to_event.year - times_to_event.year_event)*12 + (times_to_event.month - times_to_event.month_event))
times_to_event = times_to_event[times_to_event.time_to_event != 0][['numero_identificacion_paciente','time_to_event']]

times_to_event

,numero_identificacion_paciente,time_to_event
0,1000001870,32
1,1000002872,28
3,1000004036,32
4,1000005614,26
5,1000005798,34
...,...,...
281985,9994454,26
281986,999486,32
281987,999491,20
281988,9994953,19


In [59]:
full_data_with_time = full_data.merge(times_to_event, on = 'numero_identificacion_paciente', how = 'inner')
full_data_with_time[['peso','talla','perimetro_abdominal']] = full_data_with_time[['peso','talla','perimetro_abdominal']].astype(float)
full_data_with_time.hace_ejercicio = full_data_with_time.hace_ejercicio.apply(lambda x: x if x is None else x.replace('\n',''))

full_data_with_time

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
0,42995991,2022,2,2022-02-01,63,64.3,1.63,24.201136663,66.0,166.50,...,78.0,Femenino,Otros,Ninguno,1,1,0,None,0,35
1,44155333,2022,2,2022-02-01,40,75.9,1.49,34.187649205,29.5,64.30,...,88.0,Femenino,Otros,Profesional,1,1,0,None,0,34
2,447022,2022,2,2022-02-01,70,87.0,1.77,27.769797951,49.0,113.80,...,NaN,Masculino,Otros,Ninguno,1,1,0,None,0,11
3,45452118,2022,2,2022-02-01,60,60.0,1.60,23.437500000,38.0,125.00,...,NaN,Femenino,Otros,Ninguno,0,0,0,None,0,29
4,45478698,2022,2,2022-02-01,56,73.0,1.63,27.475629493,72.3,59.76,...,94.0,Femenino,Otros,Normalista,1,0,0,None,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270762,41476080,2025,1,2025-01-01,80,60.0,1.50,26.666666667,55.0,91.00,...,90.0,Femenino,Otros,Básica secundaria,0,0,0,Nunca,1,35
270763,52365346,2025,1,2025-01-01,49,120.0,1.60,46.875000000,41.3,88.65,...,NaN,Femenino,Otros,Básica secundaria,1,0,0,None,1,33
270764,63323735,2025,1,2025-01-01,60,78.9,1.62,30.064014632,55.9,71.50,...,98.0,Femenino,Otros,Profesional,0,0,0,None,1,34
270765,79511536,2025,1,2025-01-01,55,57.9,1.65,21.267217631,64.8,183.60,...,72.0,Masculino,Otros,Básica secundaria,0,0,0,None,1,19


In [60]:
pandas_gbq.to_gbq(full_data_with_time, 'Diabetes_avicena_survival.diabetes_final_3_annos', project_id='co-keralty-costomedico',if_exists = 'replace')

: 

In [60]:
full_data_with_time[full_data_with_time.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event


In [61]:
full_data[full_data.numero_identificacion_paciente == '1121827561'] #72020868

,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,HDL,LDL,trigliceridos,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes
